<a href="https://colab.research.google.com/github/KareemEzzaldin/Handwritten-Digit-Recognition-Deep-Learning-/blob/main/Handwritten_Digit_Recognition_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hand Written

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [ ]:
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

In [ ]:
X = train_df.drop('label', axis=1).values.astype(np.uint8)  # uint8 saves RAM
y = train_df['label'].values.astype(np.int64)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64
NUM_CLASSES = len(np.unique(y))

def preprocess_image(features, label=None):
    # Reshape to 28x28x1
    image = tf.reshape(features, [28, 28, 1])
    # Convert to RGB
    image = tf.image.grayscale_to_rgb(image)
    # Resize to 224x224
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    # Normalize to [0,1]
    image = tf.cast(image, tf.float32) / 255.0
    if label is None:
        return image
    return image, label

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_ds = val_ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices(test_df.values.astype(np.uint8))
test_ds = test_ds.map(lambda x: preprocess_image(x, None), num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(1).prefetch(tf.data.AUTOTUNE)

In [ ]:
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.2, min_lr=1e-7)

# Train (Feature Extraction)
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=[early_stopping, lr_reduction],
    class_weight=class_weights
)

# Fine-tuning
for layer in base_model.layers[-30:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

fine_tune_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stopping, lr_reduction],
    class_weight=class_weights
)


predictions = np.argmax(model.predict(test_ds), axis=1)

submission = pd.DataFrame({'ImageId': np.arange(1, len(predictions) + 1), 'Label': predictions})
submission.to_csv('submission.csv', index=False)
print("Saved submission.csv")


234698864/234698864 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
Epoch 1/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 322s 550ms/step - accuracy: 0.1090 - loss: 2.6695 - val_accuracy: 0.1652 - val_loss: 2.2831 - learning_rate: 1.0000e-04
Epoch 2/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 309s 553ms/step - accuracy: 0.1426 - loss: 2.2886 - val_accuracy: 0.2942 - val_loss: 2.2190 - learning_rate: 1.0000e-04
Epoch 3/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 322s 554ms/step - accuracy: 0.1997 - loss: 2.1958 - val_accuracy: 0.4802 - val_loss: 1.9804 - learning_rate: 1.0000e-04
Epoch 4/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 291s 495ms/step - accuracy: 0.2605 - loss: 2.0539 - val_accuracy: 0.6196 - val_loss: 1.7162 - learning_rate: 1.0000e-04
Epoch 5/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 294s 557ms/step - accuracy: 0.3226 - loss: 1.8917 - val_accuracy: 0.6765 - val_loss: 1.5151 - learning_rate: 1.0000e-04
Epoch 6/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 293s 558ms/step - accuracy: 0.3771 - loss: 1.7533 - val_accuracy: 0.6720 - val_loss: 1.3715 - learning_rate